# Data Processing

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
import io
from skimage.color import rgb2gray
from skimage.filters import sato
from skimage import io
import skimage.feature

## Load Data

In [2]:
def load_data(path,extension="jpg"):
    """Create a list with all the files to process into the given folder"""
    images = sorted(glob(os.path.join(path, "*."+extension)))
    return images

## Model Loss

In [3]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

## Reshape Data

In [4]:
def reshape_image(image,target_shape):
    old_image_height, old_image_width, channels = image.shape
    # create square of black
    square_dim = max(old_image_height,old_image_width)
    square_padding = np.full((square_dim,square_dim, channels), fill_value=0, dtype=np.uint8)
    # compute centering coordinate
    x_center = (square_dim - old_image_width) // 2
    y_center = (square_dim - old_image_height) // 2
    # paste the image in our new square box
    square_padding[y_center:y_center+old_image_height, 
           x_center:x_center+old_image_width] = image
    # resize the square but preserve ratio
    new_image = cv2.resize(square_padding,(target_shape[0],target_shape[1])) 
    return new_image

## Increase Vessel Contrast

In [5]:
def increase_vessel_contrast(image,model,rate=0.25,only_vessel=False):
    """ Return the image with darker vessels
    
    Given an image it will use a pre-trained vessel segmentation
    model to detect the vessels in the eye. Then it will use this
    annotation as a mask that the function will use to increase 
    the darkness of vessels according to the given rate.
    
    image : (numpy ndarray) image to process
    rate : (float) scale the darkness of vessels
    model : (keras.engine.functional.Functional) Pre-trained 
                model for vessel segmentation
    """
    x = image/255.0
    x = x.astype(np.float32)
    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255
    if only_vessel:
        y_pred = y_pred.astype(np.uint8)
        return y_pred
    """ Enhance vessels """
    z = image
    z = z.astype(np.float32)
    z -= rate*y_pred
    z[z < 0] = 0
    z[z > 255] = 255
    return z

def segment_vessels(filename):
    img = io.imread(filename)
    gray_img = rgb2gray(img)

    threshold = 0.01
    ridges = sato(gray_img, mode="reflect")
    ridges[ridges >= threshold] = 255
    ridges[ridges < threshold] = 0

    # mask to remove border of image
    hh = int(ridges.shape[0] / 2)
    hw = int(ridges.shape[1] / 2)
    rr, cc = skimage.draw.disk((hh,hw), 0.9*min(hh,hw))
    mask = np.zeros(ridges.shape, dtype=np.uint8)
    mask[rr,cc] = 1
    masked_image = ridges * mask

    return masked_image.astype(np.uint8)

## Augment Data

In [6]:
def augment_data(image, angle):
    """ return a tuple with the original image and some transformations"""
    height, width = image.shape[:2]
    center_X = width // 2 # images are squares 
    center_Y = center_X
    # rotate our image by 45 degrees around the center of the image
    M = cv2.getRotationMatrix2D((center_X, center_Y), angle, 1.0)
    rotated_image = cv2.warpAffine(image, M, (width, height))
    return [image,rotated_image]

In [7]:
def translate_data(image, tx, ty):
    width, height = image.shape[:2]
    M = np.float32([[1,0,tx],
                    [0,1,ty]])
    shifted = cv2.warpAffine(image, M, (width,height))
    return shifted

## Save Data

In [8]:
def save_data(image,output_path,i,j,gray=True,contrast_threshold=35):
    path = output_path+"/"+str(i)
    tmp = image.copy()
    tmp = tmp.astype(np.uint8)
    if not os.path.isdir(path):
        os.mkdir(path)
    gray_image = image
    if gray:
        gray_image = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
    gray_image[gray_image <= contrast_threshold] = 0
    cv2.imwrite(path+"/"+str(j)+".jpg",gray_image)

## Pipeline

In [24]:
def process_data(data_path,output_path,model_path,target_shape=(512,512),extension="jpg",vessel_ir=0.25,angle_max=6,seed=10,contrast_threshold=35,only_vessel=False):
    os.mkdir(output_path)
    np.random.seed(seed)
    # Load Images paths
    images_paths = load_data(data_path,extension)
    # Load pre-trained model
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model(model_path)
    # Process images
    for idx, image_path in tqdm(enumerate(images_paths), total=len(images_paths)):
        original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        good_shape_image = reshape_image(original_image.copy(),target_shape)
        dark_vessel_image = increase_vessel_contrast(good_shape_image.copy(),model,vessel_ir,only_vessel=only_vessel)
        angle = np.random.randint(low=-angle_max,high=angle_max)
        augmented_images = augment_data(dark_vessel_image.copy(),angle)
        for i in range(len(augmented_images)):
            final_image = augmented_images[i]
            save_data(final_image.copy(),output_path,idx,i,gray=True,contrast_threshold=contrast_threshold)

In [25]:
process_data("data/raw_ppm/","data/only_vessel",
             "/EPFL/master/ML/tmp/Vessel_Segmentation/files/model_75.h5",
            extension="ppm",angle_max=15,vessel_ir=0.25,only_vessel=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 397/397 [00:54<00:00,  7.34it/s]


# Pipeline Only Vessels

In [11]:
def create_vessel_data(data_path,output_path,model_path,target_shape=(512,512),extension="jpg",angle_max=20,translation_max=50,seed=10):
    os.mkdir(output_path)
    np.random.seed(seed)
    # Load Images paths
    images_paths = load_data(data_path,extension)
    # Load pre-trained model
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model(model_path)
    # Process images
    for idx, image_path in tqdm(enumerate(images_paths), total=len(images_paths)):
        original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        good_shape_image = reshape_image(original_image.copy(),target_shape)
        only_vessel_image = increase_vessel_contrast(good_shape_image.copy(),model,only_vessel=True)
        # Determine transformations parameters
        angle = np.random.randint(low=-angle_max,high=angle_max)
        tx = np.random.randint(low=0,high=translation_max)
        ty = np.random.randint(low=0,high=translation_max)
        augmented_images = augment_data(only_vessel_image.copy(),angle)
        augmented_images.append(translate_data(only_vessel_image.copy(),tx,ty))
        for i in range(len(augmented_images)):
            final_image = augmented_images[i]
            save_data(final_image.copy(),output_path,idx,i,gray=False)

In [12]:
create_vessel_data("data/raw_ppm/","data/vessel_only",
                   "/EPFL/master/ML/tmp/Vessel_Segmentation/files/model_75.h5",
            extension="ppm")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 397/397 [00:57<00:00,  6.91it/s]


---

# Pipeline Mask

In [9]:
def create_mask_data(data_path,output_path,target_shape=(512,512),extension=".jpg",angle_max=20,translation_max=50,seed=10):
    os.mkdir(output_path)
    np.random.seed(seed)
    # Load Images paths
    images_paths = load_data(data_path,extension)
    # Create new Images
    for idx, image_path in tqdm(enumerate(images_paths), total=len(images_paths)):
        mask_image = segment_vessels(image_path)
        good_shape_image = reshape_image(mask_image.copy()[:,:,np.newaxis],target_shape)
        angle = np.random.randint(low=-angle_max,high=angle_max)
        tx = np.random.randint(low=0,high=translation_max)
        ty = np.random.randint(low=0,high=translation_max)
        augmented_images = augment_data(good_shape_image.copy(),angle)
        augmented_images.append(translate_data(good_shape_image.copy(),tx,ty))
        for i in range(len(augmented_images)):
            final_image = augmented_images[i]
            save_data(final_image.copy(),output_path,idx,i,gray=False)

In [10]:
create_mask_data("data/raw_ppm/","data/mask",
            extension="ppm")

100%|████████████████████████████████████████████████████████████████████████████████| 397/397 [02:24<00:00,  2.74it/s]


---